### Es 1
Hai a disposizione un file `data.csv` contenente dati mensili di passeggeri con due colonne:

- `date`: data in formato `YYYY-MM` (mese/anno)
- `passengers`: numero di passeggeri per quel mese


Costruisci un modello di **regressione polinomiale** che approssima l’andamento del numero di passeggeri nel tempo.

1. Carica il dataset.
2. Convertilo in un formato numerico utilizzando una colonna `mese_numerico` che conti i mesi a partire da gennaio 1949.
3. Applica una regressione polinomiale (grado a tua scelta).
4. Calcola l’RMSE tra i valori reali e quelli predetti.
5. Visualizza i dati reali e la curva stimata con Plotly.

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

df = pd.read_csv("data.csv")
df = df.dropna()
df['passengers'] = df['passengers'].astype(int)

# Conversione della data in mese numerico
df['date'] = pd.to_datetime(df['date'], format='%Y-%m')
start_date = pd.to_datetime('1949-01')
df['mese_numerico'] = (df['date'] - start_date).dt.days // 30

X = df[['mese_numerico']]
y = df['passengers']
poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(X)

model = LinearRegression()
model.fit(X_poly, y)
y_pred = model.predict(X_poly)

# Calcolo RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"RMSE: {rmse:.2f}")

# Visualizzazione con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['date'], y=df['passengers'], mode='markers', name='Dati Reali'))
fig.add_trace(go.Scatter(x=df['date'], y=y_pred, mode='lines', name='Regressione Polinomiale'))
fig.update_layout(title='Andamento Passeggeri Aerei (1949-1960)', xaxis_title='Data', yaxis_title='Passeggeri')
fig.show()

RMSE: 44.27


### Es2. 
Costruisci una web app con Dash che permette all’utente di scegliere il grado del polinomio per adattare un modello di regressione ai dati non lineari e vedere il risultato aggiornarsi dinamicamente.


1. Genera 100 punti x tra -3 e 3.

2. Calcola ad esempio y = x³ - x + rumore.

3. Costruisci un'interfaccia Dash con:
    - uno slider per scegliere il grado del polinomio (1–10),
    - un grafico Plotly che mostra i dati e la curva stimata.

4. Usa PolynomialFeatures + LinearRegression da scikit-learn per stimare la curva

In [20]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go

x = np.linspace(-3, 3, 100)
y = x**3 - x + np.random.normal(0, 3, 100)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Slider(id='grado', min=1, max=10, step=1, value=3),
    dcc.Graph(id='grafico')
])

@app.callback(
    Output('grafico', 'figure'),
    Input('grado', 'value')
)
def aggiorna_grafico(grado):
    poly = PolynomialFeatures(grado)
    X_poly = poly.fit_transform(x.reshape(-1, 1))
    
    modello = LinearRegression()
    modello.fit(X_poly, y)
    
    y_pred = modello.predict(X_poly)
    
    punti = go.Scatter(x=x, y=y, mode='markers', name='Dati originali')
    curva = go.Scatter(x=x, y=y_pred, mode='lines', name='Curva stimata')
    
    return {'data': [punti, curva]}

app.run(debug=True)
